<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EC%B1%97%EB%B4%87_%EB%AA%A8%EB%8D%B8_%ED%95%99%EC%8A%B5(hotel_ver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRF 사용을 위한 텐서플로우 버전 조정

In [ ]:
#==과 버전 사이에 띄어쓰기 없이 그대로 사용
!pip install tensorflow==1.14.0
!pip intsall keras==2.2.4
!pip install tensorflow-gpu==1.14.0
!pip install git+https://www.github.com/keras-team/keras-contrib.git

# 사전 추가를 위한 형태소 분석기 Mecab-ko 설치

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

# 규칙정의

의도는 총 5가지

In [ ]:
#예약(예약, 평점기준 추천 등) / 변경 / 취소 / 확인 / 정보(체크인, 체크아웃, 시설 등) 

# 예약 => 호텔스 컴바인 체크인 및 아웃 날짜 기준으로 가능한 방 및 추천방 전달
# xx 지역 이용 가능한 방 있어 / xx호텔 xx지역 빙 방 있어? / 내일 xx지역 이용 가능한 방 있어?? / xx지역 평점 기준으로 방 추천해줘

# 변경 => 호텔스컴바인은 가격비교 사이트로 처리 불가. 고객이 실제 이용한 결제 페이지나 해당 호텔 연락처 전달
# xxxx 번호 방 변경해줘 / xxxx호텔에 예약한 거 방 변경해줘 / xxxx호텔 예약한 거 이용 날짜 변경해줘 / xxxx번호 방 날짜 변경할래

# 취소 => 호텔스컴바인은 가격비교 사이트로 처리 불가. 고객이 실제 이용한 결제 페이지나 해당 호텔 연락처 전달
# xxxx 번호 취소해줘 / xxxx호텔 예약 취소해줘 / xxxx 번호 취소할게요 / xxxx번호 취소

# 확인 => 호텔스컴바인은 가격비교 사이트로 처리 불가. 고객이 실제 이용한 결제 페이지나 해당 호텔 연락처 전달
# xxxx 번호 예약 확인해줘 / xxxx 번호 예약 확인

# 정보
# xxxx 호텔 체크인 or 체크아웃 시간 언제야 / xxxx호텔 입실 or 퇴실 시간 언제야 / xxxx호텔 입퇴실 시간 언제야 / xxxx호텔 체크인 및 체크아웃 시간 언제야
# xxxx 호텔 이용가능한 시설 뭐 있어 / xxxx호텔 반려견 동반 가능해? / xxxx호텔 서비스 뭐 있어?

개체명은 아래와 같이 규칙을 정한다

In [ ]:
#규칙
"""
총 8개의 태그
SEA : 검색
SUR : 서비스(체크인,체크아웃,서비스,시설 등 다 포함)
GRA : 평점
PRI : 가격
CHA : 변경
CAN : 취소
CHE : 확인, 체크
RES : 예약
DAT : 날짜(내일, 모레, 일주일 뒤, 한달 뒤)
"""

## 호텔명 및 지역명 불러오기
- hotelscombind에서 지역별로 호텔명을 가져옴
- 호텔스컴바인은 전체 리스트가 아닌, 지정한 조건(체크인, 체크아웃, 객실수, 이용객 수)에 따라 조건이 달라짐
- 해당 데이터는 4.17~4.18 객실 1개, 이용객 2명 기준으로 각 지역별로 호텔명을 크롤링
- colab에서 selenium을 이용한 크롤링에 제한사항(사용법을 잘 모름)으로 우선 local의 jupyter notebook에서 크롤링 진행  
---
- 지역명은 각 지역(서울,서귀포 등)에서 호텔스컴바인 기준으로 가져옴
  https://www.hotelscombined.co.kr/Place/Seoul.htm

In [ ]:
from google.colab import files
myfile1 = files.upload()
myfile2 = files.upload()

In [ ]:
import io
import pandas as pd 

data = pd.read_csv(io.BytesIO(myfile1['hotel_list.csv'])) #io 없이 pandas 모듈만 사용하면 에러 발생
data.head()

In [ ]:
data = data[["호텔","지역"]].copy()
data.shape

In [ ]:
data["호텔"].nunique(),data['지역'].nunique()

In [ ]:
#중복값 제거
data = data.drop_duplicates(subset = ["호텔"])
data.shape

In [ ]:
hotel_list = data["호텔"].to_list()
print(hotel_list)

In [ ]:
data_loc = pd.read_csv(io.BytesIO(myfile2['location_list.csv'])) #io 없이 pandas 모듈만 사용하면 에러 발생
print(data_loc.shape)
print("*"*30)
print(data_loc.nunique())

In [ ]:
data_loc = data_loc.drop_duplicates()
data_loc[20:30]

In [ ]:
data_loc.loc[data_loc["지역"]=="Jamsil Sports Complex","지역"] = "잠실종합운동장"
data_loc[20:30]

In [ ]:
location_list = data_loc["지역"].to_list()
print(location_list)

## 학습 데이터 만들기

### 형태소 분석기 사전 추가할 데이터 정리(호텔)

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
for i in range(30):
  print(f"[{i}번째] {hotel_list[i]} ===> {mecab.pos(hotel_list[i])}")
mecab.pos(hotel_list[0])

형태소가 부정확하게 인식하는 것에 대해서 우선 사전을 추가
- 띄어쓰기 단위로 단어를 나누고, 2글자 이상만 선택 후 사전에 추가
- 한글만 선택해서 받기

In [ ]:
import re
print(hotel_list[9])
print(re.sub(pattern="[^ㄱ-ㅎㅏ-ㅣ가-힣 ]",repl = "",string = hotel_list[9]))

In [ ]:
words_hotel = []
for i in range(len(hotel_list)):
  hotel_name = hotel_list[i]
  hotel_name = re.sub(pattern = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]",repl = "",string = hotel_name)
  for word in hotel_name.split():
    if (len(word) >= 2) & (word not in words_hotel):
      words_hotel.append(word)


In [ ]:
print("2글자 이상의 단어 개수: ", len(words_hotel))
print("-"*30)
print(words_hotel)

### 형태소 분석기 사전 추가할 데이터 정리(지역명)

In [ ]:
for i in range(len(location_list)):
  print(f"[{location_list[i]}]     ", mecab.pos(location_list[i]))

따로 추가할 필요 없어보임

### 형태소 분석기 사전 업데이트 후 재실행

In [ ]:
# 사전 추가를 위한 경로 이동 => user-dic 의 nnp(일반명사)에 단어 단어 추가
%cd /content/mecab-ko-dic-2.1.1-20180720
%ls

In [ ]:
%ls user-dic/

In [ ]:
with open("./user-dic/nnp.csv",'r') as f:
  file_data = f.readlines()

In [ ]:
file_data #이미 추가했기에 늘어남

In [ ]:
#단어 추가를 위해선 마지막 글자의 종성여부 판단이 필요
#관련 패키지 설치
!pip install jamo

In [ ]:
from jamo import h2j, j2hcj #종성 여부를 판단해주는 
#h2j => 조합형 한글로 바꾼 다음
#j2hcj => 완성형 한글로 바꾸기
j2hcj(h2j(words[0]))

In [ ]:
T_jamo = []
def get_jongsung_TF(sample_text):
  sample_text_list = list(sample_text)
  last_word = sample_text_list[-1]
  last_word_jamo_list = list(j2hcj(h2j(last_word)))
  last_jamo = last_word_jamo_list[-1]

  #마지막 종성이 모음일 경우 T
  jongsung_TF = "T"

  #마지막 종성이 자음일 경우, 즉 입력한 단어에 받침이 있는 경우 F
  if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
    jongsung_TF = "F"
  
  return jongsung_TF

In [ ]:
for i in range(10):
  print(f"글자: {words[i]:10} 받침 유무: {get_jongsung_TF(words[i])}")

In [ ]:
#사전 추가
with open("./user-dic/nnp.csv","r") as f:
  file_data = f.readlines()

for word in words:
  jongsung_TF = get_jongsung_TF(word)

  line = f"{word},,,,NNP,*,{jongsung_TF},{word},*,*,*,*,*\n"

  file_data.append(line)

In [ ]:
# 사전 작성
with open("./user-dic/nnp.csv","w") as f:
  for line in file_data:
    f.write(line)

In [ ]:
# 사전 작성 후 다시 읽기
with open("./user-dic/nnp.csv","r") as f:
  file_new = f.readlines()
file_new

In [ ]:
# 사전  추가 완료하기
%ls

In [ ]:
%ls tools

In [ ]:
!bash ./tools/add-userdic.sh

In [ ]:
!make install

In [ ]:
# 정상적으로 등록된 지 재확인
from konlpy.tag import Mecab
mecab = Mecab()

for i in range(30):
  print(f"[{i}번째] {hotel_list[i]} ===> {mecab.pos(hotel_list[i])}")
mecab.pos(hotel_list[0])

### 예약 종목 학습 데이터 만들기

In [ ]:
import numpy as np
import random
from sklearn.utils import shuffle

In [ ]:
# price_list =[] #price가 없을 경우에 약 3만개의 질문이 생성되는데, price가 여러개일 경우  질문이 백만개 단위로 생성됨, 우선 하나의 값만 적용
# for price in range(100000, 210000, 10000):
#   price_won = f"{price}원"
#   price_list.append(price)
#   price_list.append(price_won)

price_list = [100000, "100000원"]
print(price_list)

In [ ]:
def question_list(hotel_list, location_list = None, price_list = None):
  """
  hotel_list : 호텔스컴바인의 호텔 리스트 기준으로 생성
  location_list : 호텔스컴바인의 지역별 세부 지역 혹은 명소 등을 기준으로 생성
  price_list : 100000~200000 사이를 기준으로 잡고 리스트 생성

  location_list & price_list가 없을 경우 hotel_list 기준으로만 질문 생성
  """
  question = []

  #호텔 기준으로 질문 생성
  for hotel in hotel_list:
    s1 = hotel + " 이용 가능한 방 있어?"
    s2 = hotel + " 빈 방 있어?"
    s3 = hotel + " 묵을 방 있어?"
    s4 = hotel + " 방 있어?"
    s5 = hotel + " 이용 가능한 방"
    s6 = hotel + " 내일 이용 가능해?"
    s7 = hotel + " 모레 방 있어?"
    s8 = "이번 주 수요일 " + hotel + "에 방 있어?" 
    s8 = "다음 주 수요일 " + hotel + "에 방 있어?" 
    s9 = hotel + " 이용 가능한 방 있어?"
    s10 = hotel + " 빈 방 있어?"
    s11 = hotel + " 묵을 방 있어?"
    s12 = hotel + " 방 있어?"
    question = question + [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12]
  
  if location_list !=None:
    for location in location_list:
      s1 = location + " 방 있어?"
      s2 = location + " 이용 가능한 호텔 있어?"
      s3 = location + " 빈 방 있어?"
      s4 = "이번주 수요일 " + location + "에 방 있어?"
      s5 = "다음주 수요일 " + location + "에 방 있어?"
      s6 = location + " 빈 방 알려줘"
      s7 = location + " 평점 기준으로 추천해 줘"
      s8 = location + " 평점 높은 곳이 어디야"
      s9 = location + " 추천해줘"
      question = question + [s1,s2,s3,s4,s5,s6,s7,s8,s9]
  
  if price_list != None:
    list_add_prices = question.copy()
    for price in price_list:
      for add_price in list_add_prices:
        s0 = f"{price} 이하 " + add_price
        s1 = f"{price} 밑으로 " + add_price
        question = question + [s0, s1]
  
  return question
question = question_list(hotel_list, location_list, price_list)

In [ ]:
reservation = question
len(reservation)

In [ ]:
random.sample(reservation, 20)

In [ ]:
reservation[-1]

In [ ]:
test_result = question_list(["신라 호텔"], ["서울 지역"], ["100000원"])
test_result

In [ ]:
tokenized_test_result = []
for sample_keyword_question in test_result:
  tokenized_test_result.append(mecab.morphs(sample_keyword_question))

In [ ]:
for sample in tokenized_test_result:
  print(sample)

향후 필요한 키워드와 불필요한 키워드를 분류해야 하므로 현재 인텐트의 불필요한 단어를 정리

In [ ]:
reservation_stopwords_list = set([item for sample in tokenized_test_result for item in sample if item not in ['신라','호텔','내일','모레','주','수요일','서울','평점','추천','100000']])
print(reservation_stopwords_list)

In [ ]:
reservation_data = {'question' : reservation, 'intent' : "예약"}
reservation_df = pd.DataFrame(reservation_data, columns = ["question","intent"])

In [ ]:
reservation_df.sample(20)

### 변경 종목 학습 데이터 만들기